In [157]:
import minio
import ldap3
from dotenv import dotenv_values
import logging

In [158]:
ldapSetupLogger = logging.getLogger('ldapSetup')

In [159]:
configValues = dotenv_values(dotenv_path="./setup.env")

# create an admin connection to ldap3
ldapServerURI = f"ldap://{configValues['FAIRSCAPE_LDAP_HOST']}:{configValues['FAIRSCAPE_LDAP_PORT']}" 
#ldapServer = ldap3.Server(ldapServerURI)

# for tests
ldapServerURI = "ldap://localhost:1389"


ldapBaseDN = configValues['FAIRSCAPE_LDAP_BASE_DN']
ldapUsersDN = configValues['FAIRSCAPE_LDAP_USERS_DN']
ldapGroupsDN = configValues['FAIRSCAPE_LDAP_GROUPS_DN']

configAdminDN = configValues['FAIRSCAPE_LDAP_CONFIG_ADMIN_DN']
configAdminPassword = configValues['FAIRSCAPE_LDAP_CONFIG_ADMIN_PASSWORD']

adminDN = configValues['FAIRSCAPE_LDAP_ADMIN_DN']
adminPassword = configValues['FAIRSCAPE_LDAP_ADMIN_PASSWORD']


In [131]:
adminDN

'cn=admin,dc=fairscape,dc=net'

In [110]:
configValues['FAIRSCAPE_LDAP_CONFIG_ADMIN_DN']

'cn=configadmin,cn=config'


`docker run --restart always --name openldap --env-file ./deploy/ldap.env -p 1389:1389 bitnami/openldap:latest`

LDAP_SKIP_DEFAULT_TREE="no"

In [160]:
# exceptions

# connect as the config admin
class LDAPConnectionError(Exception):
	def __init__(self,exception: Exception | None=None, message: str="failed to connect to ldap"):

		self.message = message
		self.exception = exception
		super().__init__(self.message)


In [161]:
# helper utilities

def connectLDAP(userDN, userPassword):
	try:
		server = ldap3.Server(ldapServerURI, get_info=ldap3.ALL)
		connection = ldap3.Connection(server,          
			user=userDN, 
			password=userPassword,
			lazy=False
			) 
		bind_response = connection.bind()
		if bind_response:
			ldapSetupLogger.info(f"msg: 'connected to LDAP server'\tserver: '{ldapServerURI}'\tuserDN: '{userDN}'")
		return connection
	
	except ldap3.core.exceptions.LDAPBindError as bindError:
		ldapSetupLogger.error(f"msg: 'ldap bind error'\tserver: '{ldapServerURI}'\tuserDN: '{userDN}'")
		raise LDAPConnectionError(
			exception=bindError, 
			message="ldap bind error"
			) 

	except ldap3.core.exceptions.LDAPSocketOpenError as connError:
		ldapSetupLogger.error(f"msg: 'ldap failed to open connection'\tserver: '{ldapServerURI}'\tuserDN: '{userDN}'")
		raise LDAPConnectionError(
			exception=connError, 
			message=f"ldap connection error: failed to open connection at {ldapServerURI}"
			) 

	except ldap3.core.exceptions.LDAPException as ldapException:
		ldapSetupLogger.error(f"msg: 'ldap exception'\tserver: '{ldapServerURI}'\tuserDN: '{userDN}'")
		raise LDAPConnectionError(
			exception=ldapException, 
			message=f"ldap error: {str(ldapException)}"
			)

In [162]:
def setupOverlay():
    # connect as the config admin
    configAdminConnection =connectLDAP(
        userDN=configAdminDN,
        userPassword=configAdminPassword
        )

    # add the module list to the config database 
    addModuleList = configAdminConnection.add(
        dn="cn=module,cn=config",
        attributes={
            "objectClass": "olcModuleList",
            "olcModuleLoad": [ "memberof.so", "refint.so"],
            "olcModulePath": "/opt/bitnami/openldap/lib/openldap"
        }
    )

    if not addModuleList:
        ldapSetupLogger.error(msg="msg: failed to add module list to config database")
        raise Exception("Setup Failed")

    configAdminConnection.rebind()

    try:
        ldap3.ObjectDef(['olcMemberOf'], configAdminConnection)

    except ldap3.core.exceptions.LDAPSchemaError as schemaError:
        ldapSetupLogger.error("failed to find olcMemberOf schema")
        raise Exception("Setup Failed")


    memberOfOverlayDN="olcOverlay={0}memberof,olcDatabase={2}mdb,cn=config"
    memberOfOverlayAttributes={
        "objectClass": [ "olcOverlayConfig", "olcMemberOf"],
        "olcOverlay": "memberof",
        "olcMemberOfRefInt": "TRUE",
    }

    addOverlay = configAdminConnection.add(
        dn=memberOfOverlayDN,
        attributes=memberOfOverlayAttributes,
    )


    configAdminConnection.rebind()

    if not addOverlay:
        overlayAppliedResult = configAdminConnection.search(
            search_base="olcDatabase={2}mdb,cn=config",
            search_scope=ldap3.SUBTREE,
            search_filter="(objectClass=olcOverlayConfig)"
        )

        if not overlayAppliedResult:
            ldapSetupLogger.error("failed to find applied memberOf overlay")
            raise Exception("ldap setup failure")

In [163]:
setupOverlay()

In [164]:
# add user
def addFairscapeUser(
    ldapConnection, 
    userCN: str, 
    userSN: str, 
    userGN: str, 
    userMail: str,
    userPassword: str,
):

    return ldapConnection.add(
        dn=f"cn={userCN},ou=users,{ldapBaseDN}",
        attributes={
            "objectClass": ["inetOrgPerson", "Person"],
            "cn": userCN,
            "sn": userSN,
            "gn": userGN,
            "mail": userMail,
            "userPassword": userPassword
        }    
    )

In [ ]:
def setupLDAPUsers():

	# form admin connection to main database
	adminConnection = connectLDAP(
		adminDN,
		adminPassword
	)

	# add groups OU
	adminConnection.add(
		dn="ou=groups,dc=fairscape,dc=net",
		attributes={
			"objectClass": "organizationalUnit", 
			"ou": "groups", 
			"description": "organizational unit of fairscape groups"
			}
	) 

	userList= loadUsers()
	groupList= loadGroups()

	# create users
	createUsers(adminConnection, userList)
	createGroups(adminConnection, groupList)


# iterate over user data
def loadUsers():
	with open("./data/user_data.csv", "r") as csvFile:

		# ignore the first line
		userData = csvFile.read()

	userList = []

	lines = userData.splitlines()
	for userLine in lines[1::]:
		userData = userLine.replace(" ", "").split(",")

		user = {
			"firstName": userData[0],
			"lastName": userData[1],
			"dn": userData[2],
			"email": userData[3],
			"password": userData[4]
		}

		userList.append(user)
	return userList


def createUsers(passedLDAPConnection, userList):
	# for all users
	for userElem in userList[1::]:
		addSuccess = addFairscapeUser(
			ldapConnection=passedLDAPConnection, 
			userCN=userElem['dn'],
			userSN=userElem['lastName'],
			userGN=userElem['firstName'],
			userMail=userElem['email'],
			userPassword=userElem['password']
			)
		ldapSetupLogger.info(f"msg: added user\tsuccess: {addSuccess}\tuser: {userElem['dn']}")


def loadGroups():
	with open("./data/group_data.csv", "r") as csvFile:

		# ignore the first line
		groupData = csvFile.read()

	groupList = []

	lines = groupData.splitlines()
	for groupLine in lines[1::]:
		groupData = groupLine.replace(" ", "").split(",")

		group = {
			"dn": groupData[0],
			"members": [ f"cn={groupMember},ou=users,{ldapBaseDN}" for groupMember in groupData[1].split(";")],
		}

		groupList.append(group)
	return groupList


# add all groups
def createGroups(ldapConnection, groupList):

	for groupElem in groupList:
		addGroup = ldapConnection.add(
			dn=f"cn={groupElem['dn']},ou=groups,dc=fairscape,dc=net",
			attributes={
			"objectClass": "groupOfNames", 
				"cn": groupElem['dn'], 
				"member": groupElem['members']
			}
		)
		ldapSetupLogger.info(f"added group\tsuccess:{addGroup}\tgroup: {groupElem['dn']}")


# spot test that members have the memberOf
def spotCheck(ldapConnection):
	ldapConnection.rebind()

	ldapConnection.search(
		search_base="ou=users,dc=fairscape,dc=net",	
		search_scope=ldap3.SUBTREE,
		search_filter='(objectClass=*)',
		attributes=['*', 'memberOf']
	)

	ldapConnection.entries

True

In [136]:
help(adminConnection.add)

Help on method add in module ldap3.core.connection:

add(dn, object_class=None, attributes=None, controls=None) method of ldap3.core.connection.Connection instance
    Add dn to the DIT, object_class is None, a class name or a list
    of class names.
    
    Attributes is a dictionary in the form 'attr': 'val' or 'attr':
    ['val1', 'val2', ...] for multivalued attributes



In [138]:
help(addFairscapeUser)

Help on function addFairscapeUser in module __main__:

addFairscapeUser(ldapConnection, userCN: str, userSN: str, userGN: str, userMail: str, userPassword: str)
    # add user



In [ ]:
str

[{'dn': 'admins', 'members': ['cn=johndoe,ou=users,dc=fairscape,dc=net']},
 {'dn': 'fictional',
  'members': ['cn=johndoe,ou=users,dc=fairscape,dc=net',
   'cn=maxheadroom,ou=users,dc=fairscape,dc=net']},
 {'dn': 'adultswim',
  'members': ['cn=tim,ou=users,dc=fairscape,dc=net',
   'cn=eric,ou=users,dc=fairscape,dc=net',
   'cn=ericandre,ou=users,dc=fairscape,dc=net']}]

In [145]:
groupElem = group

True

In [176]:
adminConnection.entries

[DN: ou=users,dc=fairscape,dc=net - STATUS: Read - READ TIME: 2024-10-18T13:41:13.172845
     objectClass: organizationalUnit
     ou: users,
 DN: cn=tim,ou=users,dc=fairscape,dc=net - STATUS: Read - READ TIME: 2024-10-18T13:41:13.172845
     cn: tim
     givenName: tim
     mail: tim_heidecker@example.org
     memberOf: cn=adultswim,ou=groups,dc=fairscape,dc=net
     objectClass: inetOrgPerson
                  person
     sn: heidecker
     userPassword: b'timanderic',
 DN: cn=eric,ou=users,dc=fairscape,dc=net - STATUS: Read - READ TIME: 2024-10-18T13:41:13.172845
     cn: eric
     givenName: eric
     mail: eric_weirheim@example.rog
     memberOf: cn=adultswim,ou=groups,dc=fairscape,dc=net
     objectClass: inetOrgPerson
                  person
     sn: weirheim
     userPassword: b'ericandtim',
 DN: cn=admins,ou=users,dc=fairscape,dc=net - STATUS: Read - READ TIME: 2024-10-18T13:41:13.172845
     cn: admins
     member: cn=fairscapeUser,ou=users,dc=fairscape,dc=net
     objectCla

In [ ]:
# if already exists raises
# ldap3.core.exceptions.LDAPEntryAlreadyExistsResult

In [ ]:
def getConfig():
	configValues = dotenv_values(dotenv_path="./setup.env")